In [1]:
# Импортируем необходимые библиотеки
from neo4j import GraphDatabase

In [2]:
# Напишем класс-коннектор, для удобного соединения с neo4j
class NeoConnection:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
        
    def close(self):
        self.driver.close()
        
    def query(self, query, db=None):
        assert self.driver is not None, 'No driver'
        session = None
        response = None
        try:
            session = self.driver.session(database=db) if db is not None else self.driver.session()
            response = list(session.run(query))
            
        except Exception as ex:
            print('Query failed:\n', ex)
        finally:
            if session is not None:
                session.close()
        return response

In [3]:
# Подключаемся
connect = NeoConnection(uri='bolt://localhost:7687', user='neo4j', password='ms_test26284')

In [4]:
# Передаем запрос
connect.query('CREATE OR REPLACE DATABASE mstest')

[]

In [5]:
# Считываем данные из csv файла, предварительно скачанного с яндекс.диска и помещенного в папку импорта. 
# Это сделано (скачано и помещено в папку импорта) для того, чтобы не возиться с изменениями конфигурации neo4j
query_string = '''
LOAD CSV FROM 
"file:///data_test.csv"
AS line FIELDTERMINATOR ';'
WITH line WHERE toInteger(line[0]) IS NOT NULL
MERGE (i:Ids {id: toInteger(line[0])})
MERGE (e1:Event1 {FNMember: line[1]}) 
MERGE (e2:Event2 {FNMember: line[2]})
CREATE (e1)-[:ids]->(i)
CREATE (e2)-[:ids]->(i)
'''
connect.query(query_string, db='mstest')

[]

In [6]:
# Достаем из бд те id у которых больше 2 связей 
query_string1 = '''
MATCH (p)-[r]->(i:Ids)
WITH i, count(*) as cnt
WHERE cnt > 2
RETURN properties(i)'''
req = connect.query(query_string1, db='mstest')
print(req)

[<Record properties(i)={'id': 70049}>, <Record properties(i)={'id': 92995}>, <Record properties(i)={'id': 117280}>, <Record properties(i)={'id': 177407}>, <Record properties(i)={'id': 358194}>, <Record properties(i)={'id': 390312}>, <Record properties(i)={'id': 523688}>, <Record properties(i)={'id': 551592}>, <Record properties(i)={'id': 613539}>, <Record properties(i)={'id': 716489}>, <Record properties(i)={'id': 765223}>, <Record properties(i)={'id': 850472}>, <Record properties(i)={'id': 873359}>, <Record properties(i)={'id': 938764}>, <Record properties(i)={'id': 985851}>]


In [7]:
# Избавляемся от лишней информации и оставляем только id
ids = []
for i in req:
    ids.append(i.value()['id'])
print(ids)

[70049, 92995, 117280, 177407, 358194, 390312, 523688, 551592, 613539, 716489, 765223, 850472, 873359, 938764, 985851]


In [8]:
# Достаем из бд все графы с найденными выше id
query_string2 = f'''
MATCH p=()-[]->(i:Ids)
WHERE i.id IN {ids}
RETURN nodes(p)
'''
req1 = connect.query(query_string2, db='mstest')
print(req1[0])

<Record nodes(p)=[<Node element_id='4:2b23b6e5-ff0a-44c0-83e2-e396b4b17e6e:1001' labels=frozenset({'Event1'}) properties={'FNMember': 'Яшина Полина Евгеньевна'}>, <Node element_id='4:2b23b6e5-ff0a-44c0-83e2-e396b4b17e6e:878' labels=frozenset({'Ids'}) properties={'id': 70049}>]>


In [9]:
# Функция для получения данных из бд
def rest(fn: str):
    import json
    query_string = '''
    MATCH (f)-[i]->(id) 
    WHERE f.FNMember = '%(fn)s'
    RETURN {id: id.id, ФИО_Участника_События: f.FNMember} as FN
    ''' % {"fn":fn}
    try:
        req = connect.query(query_string, db='mstest')
    except Exception as ex:
        print('Something went wrong:\n', ex)
    req1 = [record.value() for record in req]
    req2 = json.dumps(req1, indent=4, ensure_ascii=False)
    return req2
print(rest('Зимнухова Карина Даниловна'))

[
    {
        "ФИО_Участника_События": "Зимнухова Карина Даниловна",
        "id": 132775
    },
    {
        "ФИО_Участника_События": "Зимнухова Карина Даниловна",
        "id": 241336
    },
    {
        "ФИО_Участника_События": "Зимнухова Карина Даниловна",
        "id": 346700
    },
    {
        "ФИО_Участника_События": "Зимнухова Карина Даниловна",
        "id": 348716
    },
    {
        "ФИО_Участника_События": "Зимнухова Карина Даниловна",
        "id": 504411
    }
]
